In [1]:
import pandas as pd
import numpy as np
import time
#显示所有列
pd.set_option('display.max_columns', None)
start_time=time.time()
print('start_time:',start_time)
'''
Load Data
'''
IDIR = 'G:\\bigdata\\badou\\00-data//'
priors = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
    'order_id': np.int32,
    'product_id': np.uint16,
    'add_to_cart_order': np.int16,
    'reordered': np.int8})
# 用户倒数第二天购买的商品
train = pd.read_csv(IDIR + 'order_products__train.csv', dtype={
    'order_id': np.int32,
    'product_id': np.uint16,
    'add_to_cart_order': np.int16,
    'reordered': np.int8})
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
    'order_id': np.int32,
    'user_id': np.int32,
    'eval_set': 'object',
    'order_number': np.int16,
    'order_dow': np.int8,
    'order_hour_of_day': np.int8,
    'days_since_prior_order': np.float32})
products = pd.read_csv(IDIR + 'products.csv', dtype={
    'product_id': np.uint16,
    'order_id': np.int32,
    'aisle_id': np.uint8,
    'department_id': np.uint8},
                       usecols=['product_id', 'aisle_id', 'department_id'])

print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))


'''
Feature Process
'''
# 1. product feature
print('computing product f')
prods = pd.DataFrame()
# 产品销量
prods['orders'] = priors.groupby(priors.product_id).size().astype(np.int32)
# 产品再次购买量
prods['reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)
# 产品再次购买比例
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)
products = products.join(prods, on='product_id')
products.set_index('product_id', drop=False, inplace=True)
del prods

start_time: 1597473903.89519
priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered
computing product f


In [ ]:
orders.head()